Block 1: Problem Description

In [6]:
# Block 1: Problem Description (according to the paper)

import math
import random
import copy

# --- Constants and Enums ---
NODE_TYPE_WAREHOUSE = "WAREHOUSE"
NODE_TYPE_STORE = "STORE"
NODE_TYPE_SATELLITE = "SATELLITE"

# --- Helper Functions ---
def euclidean_distance(node1_coords, node2_coords):
    return math.sqrt((node1_coords[0] - node2_coords[0])**2 + \
                     (node1_coords[1] - node2_coords[1])**2)

# --- Core Classes ---
class ProductOrderType:
    def __init__(self, type_id, volume):
        self.id = type_id
        self.volume = volume # Vr for products, Vb for online orders

class Node:
    def __init__(self, node_id, node_type, x, y, service_time=0):
        self.id = node_id
        self.type = node_type # STORE, SATELLITE, WAREHOUSE
        self.coords = (x, y)
        self.demands = {} # Key: product_type_id (str), Value: quantity (int)
        self.delivery_time_window = None # For stores: [earliest_arrival, latest_arrival (li)] (hard)
                                         # For satellites: [earliest_arrival, latest_arrival (li)] (soft)
        self.service_time = service_time # Oi

    def add_demand(self, product_type_id, quantity, product_type_obj=None):
        self.demands[product_type_id] = {
            "quantity": quantity,
            "obj": product_type_obj # Store the ProductOrderType object for easy volume access
        }

    def get_total_demand_volume(self):
        total_vol = 0
        for pt_id, demand_info in self.demands.items():
            total_vol += demand_info["quantity"] * demand_info["obj"].volume
        return total_vol

    def __repr__(self):
        return f"Node({self.id}, {self.type}, Demands: {sum(d['quantity'] for d in self.demands.values())} items, TW: {self.delivery_time_window})"

class Vehicle:
    def __init__(self, vehicle_id, capacity_volume):
        self.id = vehicle_id
        self.capacity_volume = capacity_volume # V*

    def __repr__(self):
        return f"Vehicle({self.id}, Capacity: {self.capacity_volume})"

class ProblemInstance:
    def __init__(self, pu_penalty_soft_tw, ps_extra_cost_split):
        self.nodes = {} # Key: node_id, Value: Node object
        self.warehouse = None
        self.stores = []
        self.satellites = []
        self.vehicles = [] # Fleet VF
        self.product_order_types = {} # Key: type_id, Value: ProductOrderType object

        # Costs
        self.Pu_penalty_soft_tw = pu_penalty_soft_tw # Unit penalty for delayed delivery to satellites
        self.Ps_extra_cost_split = ps_extra_cost_split # Extra cost for split delivery

    def add_node(self, node):
        self.nodes[node.id] = node
        if node.type == NODE_TYPE_WAREHOUSE:
            self.warehouse = node
        elif node.type == NODE_TYPE_STORE:
            self.stores.append(node)
        elif node.type == NODE_TYPE_SATELLITE:
            self.satellites.append(node)

    def add_vehicle(self, vehicle):
        self.vehicles.append(vehicle)

    def add_product_order_type(self, pt_obj):
        self.product_order_types[pt_obj.id] = pt_obj

    def get_node(self, node_id):
        return self.nodes.get(node_id)

    def get_all_customer_nodes(self): # Stores and Satellites
        return self.stores + self.satellites

    def get_distance(self, node_id1, node_id2):
        # Assuming distance = travel_time for simplicity as per paper (dij = tij)
        node1 = self.get_node(node_id1)
        node2 = self.get_node(node_id2)
        return euclidean_distance(node1.coords, node2.coords)

    def __repr__(self):
        return (f"ProblemInstance(Nodes: {len(self.nodes)}, "
                f"Vehicles: {len(self.vehicles)}, Pu: {self.Pu_penalty_soft_tw}, Ps: {self.Ps_extra_cost_split})")

# --- Example of setting up a problem (mimicking FVRPOR) ---
def setup_simple_instance():
    """
    Creates a small problem instance based on the paper's description.
    This is for FVRPOR (First echelon: Warehouse -> Stores/Satellites)
    """
    # Costs from paper (example values, not specified in paper for instance gen)
    Pu = 10 # Unit penalty cost for delivery time (satellites)
    Ps = 10 # Extra cost caused by split delivery

    problem = ProblemInstance(pu_penalty_soft_tw=Pu, ps_extra_cost_split=Ps)

    # Product/Order Types (R for products, B for online orders)
    # Assuming 2 product types and 2 online order types
    pt1 = ProductOrderType("P1", volume=2) # Product for stores
    pt2 = ProductOrderType("P2", volume=3) # Product for stores
    ot1 = ProductOrderType("O1", volume=1) # Online order for satellites
    ot2 = ProductOrderType("O2", volume=2.5) # Online order for satellites
    problem.add_product_order_type(pt1)
    problem.add_product_order_type(pt2)
    problem.add_product_order_type(ot1)
    problem.add_product_order_type(ot2)

    # Central Warehouse (Node 0)
    warehouse = Node("W0", NODE_TYPE_WAREHOUSE, x=25, y=25, service_time=0)
    # Warehouse typically has a time window like [0, large_number] for departures/returns
    warehouse.delivery_time_window = [0, 1000] # Operational window
    problem.add_node(warehouse)

    # Stores (S) - Hard time windows
    store1 = Node("S1", NODE_TYPE_STORE, x=10, y=10, service_time=10)
    store1.add_demand("P1", 5, pt1)
    store1.add_demand("P2", 3, pt2)
    store1.delivery_time_window = [0, 100] # Arrive by time 100 (li)
    problem.add_node(store1)

    store2 = Node("S2", NODE_TYPE_STORE, x=40, y=40, service_time=12)
    store2.add_demand("P1", 2, pt1)
    store2.delivery_time_window = [0, 120]
    problem.add_node(store2)

    # Satellites (M) - Soft time windows
    satellite1 = Node("M1", NODE_TYPE_SATELLITE, x=15, y=35, service_time=8)
    satellite1.add_demand("O1", 10, ot1)
    satellite1.add_demand("O2", 4, ot2)
    satellite1.delivery_time_window = [0, 90] # Target by time 90 (li)
    problem.add_node(satellite1)

    # Vehicles (VF) - Homogeneous fleet
    # Paper: |VF| >= (Total demand volume) / V*
    # For this example, let's add 2 vehicles
    vehicle_capacity = 50 # V*
    problem.add_vehicle(Vehicle("V1", vehicle_capacity))
    problem.add_vehicle(Vehicle("V2", vehicle_capacity))
    # A more formal calculation for |VF|:
    # total_system_demand_vol = sum(n.get_total_demand_volume() for n in problem.get_all_customer_nodes())
    # num_vehicles_needed = math.ceil(total_system_demand_vol / vehicle_capacity)
    # print(f"Calculated min vehicles needed: {num_vehicles_needed}")
    # for i in range(num_vehicles_needed):
    #     problem.add_vehicle(Vehicle(f"V{i+1}", vehicle_capacity))


    return problem

print("--- Block 1: Problem Description Setup ---")
simple_problem_instance = setup_simple_instance()
print(simple_problem_instance)
for node in simple_problem_instance.nodes.values():
    print(f"  {node} -> Total Volume: {node.get_total_demand_volume()}")
for vehicle in simple_problem_instance.vehicles:
    print(f"  {vehicle}")
print("-" * 40)

--- Block 1: Problem Description Setup ---
ProblemInstance(Nodes: 4, Vehicles: 2, Pu: 10, Ps: 10)
  Node(W0, WAREHOUSE, Demands: 0 items, TW: [0, 1000]) -> Total Volume: 0
  Node(S1, STORE, Demands: 8 items, TW: [0, 100]) -> Total Volume: 19
  Node(S2, STORE, Demands: 2 items, TW: [0, 120]) -> Total Volume: 4
  Node(M1, SATELLITE, Demands: 14 items, TW: [0, 90]) -> Total Volume: 20.0
  Vehicle(V1, Capacity: 50)
  Vehicle(V2, Capacity: 50)
----------------------------------------


Block 2: The ALNS Framework

In [8]:
# Block 1: Problem Description Setup (Keep as is)
import math
import random
import copy

# --- Constants and Enums (from Block 1) ---
NODE_TYPE_WAREHOUSE = "WAREHOUSE"
NODE_TYPE_STORE = "STORE"
NODE_TYPE_SATELLITE = "SATELLITE"

# --- Helper Functions (from Block 1) ---
def euclidean_distance(node1_coords, node2_coords):
    return math.sqrt((node1_coords[0] - node2_coords[0])**2 + \
                     (node1_coords[1] - node2_coords[1])**2)

# --- Core Classes (from Block 1 - Assuming ProductOrderType, Node, Vehicle, ProblemInstance are defined) ---
# Re-pasting ProblemInstance and Node for completeness if running standalone
class ProductOrderType:
    def __init__(self, type_id, volume):
        self.id = type_id
        self.volume = volume

class Node:
    def __init__(self, node_id, node_type, x, y, service_time=0):
        self.id = node_id
        self.type = node_type
        self.coords = (x, y)
        self.demands = {}
        self.delivery_time_window = None
        self.service_time = service_time

    def add_demand(self, product_type_id, quantity, product_type_obj=None):
        self.demands[product_type_id] = {
            "quantity": quantity,
            "obj": product_type_obj
        }

    def get_total_demand_volume(self):
        total_vol = 0
        for pt_id, demand_info in self.demands.items():
            if demand_info["obj"]: # Ensure obj is present
                 total_vol += demand_info["quantity"] * demand_info["obj"].volume
        return total_vol
    
    def __repr__(self):
        return f"Node({self.id}, {self.type}, TW: {self.delivery_time_window})"


class Vehicle:
    def __init__(self, vehicle_id, capacity_volume):
        self.id = vehicle_id
        self.capacity_volume = capacity_volume

    def __repr__(self):
        return f"Vehicle({self.id}, Capacity: {self.capacity_volume})"

class ProblemInstance:
    def __init__(self, pu_penalty_soft_tw, ps_extra_cost_split):
        self.nodes = {}
        self.warehouse = None
        self.stores = []
        self.satellites = []
        self.vehicles = []
        self.product_order_types = {}
        self.Pu_penalty_soft_tw = pu_penalty_soft_tw
        self.Ps_extra_cost_split = ps_extra_cost_split

    def add_node(self, node):
        self.nodes[node.id] = node
        if node.type == NODE_TYPE_WAREHOUSE:
            self.warehouse = node
        elif node.type == NODE_TYPE_STORE:
            self.stores.append(node)
        elif node.type == NODE_TYPE_SATELLITE:
            self.satellites.append(node)

    def add_vehicle(self, vehicle):
        self.vehicles.append(vehicle)

    def add_product_order_type(self, pt_obj):
        self.product_order_types[pt_obj.id] = pt_obj

    def get_node(self, node_id):
        return self.nodes.get(node_id)

    def get_all_customer_nodes(self):
        return self.stores + self.satellites

    def get_distance(self, node_id1, node_id2):
        node1 = self.get_node(node_id1)
        node2 = self.get_node(node_id2)
        if not node1 or not node2: # Basic check
            # print(f"Error: Node not found for distance calc: {node_id1} or {node_id2}")
            return float('inf')
        return euclidean_distance(node1.coords, node2.coords)


# --- Example setup (from Block 1) ---
def setup_simple_instance():
    Pu = 10
    Ps = 10
    problem = ProblemInstance(pu_penalty_soft_tw=Pu, ps_extra_cost_split=Ps)

    pt1 = ProductOrderType("P1", volume=2)
    pt2 = ProductOrderType("P2", volume=3)
    ot1 = ProductOrderType("O1", volume=1)
    ot2 = ProductOrderType("O2", volume=2.5)
    problem.add_product_order_type(pt1)
    problem.add_product_order_type(pt2)
    problem.add_product_order_type(ot1)
    problem.add_product_order_type(ot2)

    warehouse = Node("W0", NODE_TYPE_WAREHOUSE, x=25, y=25, service_time=0)
    warehouse.delivery_time_window = [0, 1000]
    problem.add_node(warehouse)

    store1 = Node("S1", NODE_TYPE_STORE, x=10, y=10, service_time=10) # Total Vol: 5*2 + 3*3 = 19
    store1.add_demand("P1", 5, pt1)
    store1.add_demand("P2", 3, pt2)
    store1.delivery_time_window = [0, 100]
    problem.add_node(store1)

    store2 = Node("S2", NODE_TYPE_STORE, x=40, y=40, service_time=12) # Total Vol: 2*2 = 4
    store2.add_demand("P1", 2, pt1)
    store2.delivery_time_window = [0, 120]
    problem.add_node(store2)

    satellite1 = Node("M1", NODE_TYPE_SATELLITE, x=15, y=35, service_time=8) # Total Vol: 10*1 + 4*2.5 = 20
    satellite1.add_demand("O1", 10, ot1)
    satellite1.add_demand("O2", 4, ot2)
    satellite1.delivery_time_window = [0, 90]
    problem.add_node(satellite1)

    vehicle_capacity = 50
    problem.add_vehicle(Vehicle("V1", vehicle_capacity))
    problem.add_vehicle(Vehicle("V2", vehicle_capacity))
    return problem


# Block 2: ALNS Framework (REVISED)
class Solution:
    def __init__(self, problem_instance):
        self.problem_instance = problem_instance
        self.routes = [[] for _ in problem_instance.vehicles]
        self.cost = float('inf')
        self.unassigned_nodes = [node.id for node in problem_instance.get_all_customer_nodes()]

    def calculate_cost(self):
        total_distance_cost = 0
        total_penalty_cost = 0
        total_split_delivery_cost = 0
        
        node_visits_by_vehicle = {node.id: set() for node in self.problem_instance.get_all_customer_nodes()}
        # Ensure warehouse is in the dict to avoid KeyErrors if a route is just W0->W0 (should not happen)
        if self.problem_instance.warehouse:
            node_visits_by_vehicle[self.problem_instance.warehouse.id] = set()


        for vehicle_idx, route_tuples_original in enumerate(self.routes):
            if not route_tuples_original:
                continue

            current_node_id_sim = self.problem_instance.warehouse.id
            current_time_sim = self.problem_instance.warehouse.delivery_time_window[0] # Start time from warehouse

            first_customer_node_id_in_route = route_tuples_original[0][0]
            dist_to_first = self.problem_instance.get_distance(current_node_id_sim, first_customer_node_id_in_route)
            total_distance_cost += dist_to_first
            current_time_sim += dist_to_first
            
            for i, (node_id, delivered_items, _, _) in enumerate(route_tuples_original):
                node_obj = self.problem_instance.get_node(node_id)
                node_visits_by_vehicle[node_id].add(vehicle_idx) # Track for split cost

                arrival_time_at_node = current_time_sim
                
                effective_arrival_time = arrival_time_at_node
                # Wait if arriving before node's earliest time window
                if node_obj.delivery_time_window and node_obj.delivery_time_window[0] > arrival_time_at_node:
                     effective_arrival_time = node_obj.delivery_time_window[0]
                
                if node_obj.type == NODE_TYPE_SATELLITE:
                    _, latest_time = node_obj.delivery_time_window
                    if arrival_time_at_node > latest_time: # Penalty applies to actual arrival, not effective arrival
                        volume_this_visit = sum(
                            qty * self.problem_instance.product_order_types[pt_id].volume
                            for pt_id, qty in delivered_items.items()
                        )
                        delay = arrival_time_at_node - latest_time
                        total_penalty_cost += self.problem_instance.Pu_penalty_soft_tw * delay * volume_this_visit

                departure_time_from_node = effective_arrival_time + node_obj.service_time
                current_time_sim = departure_time_from_node
                
                if i + 1 < len(route_tuples_original):
                    next_node_id = route_tuples_original[i+1][0]
                    dist_to_next = self.problem_instance.get_distance(node_id, next_node_id)
                    total_distance_cost += dist_to_next
                    current_time_sim += dist_to_next
                else: # Last customer, travel back to warehouse
                    dist_to_warehouse = self.problem_instance.get_distance(node_id, self.problem_instance.warehouse.id)
                    total_distance_cost += dist_to_warehouse
                    current_time_sim += dist_to_warehouse # Arrival time back at warehouse
        
        # Calculate split delivery cost
        for node_id_customer in self.problem_instance.get_all_customer_nodes(): # Only for customer nodes
            num_distinct_vehicle_visits = len(node_visits_by_vehicle[node_id_customer.id])
            if num_distinct_vehicle_visits > 1:
                total_split_delivery_cost += self.problem_instance.Ps_extra_cost_split * (num_distinct_vehicle_visits - 1)
        
        unassigned_penalty = 0
        if self.unassigned_nodes: # If the list itself is not empty.
            # The actual check for whether demand is met is in is_feasible.
            # Here, we just penalize if the *list* unassigned_nodes is populated.
            unassigned_penalty = len(self.unassigned_nodes) * 1_000_000 
        
        self.cost = total_distance_cost + total_penalty_cost + total_split_delivery_cost + unassigned_penalty
        return self.cost

    def is_feasible(self, debug_feasibility=False):
        # 1. Demand Satisfaction (Primary Check)
        delivered_quantities_at_nodes = {
            node.id: {pt_id: 0 for pt_id in node.demands}
            for node in self.problem_instance.get_all_customer_nodes()
        }

        for route in self.routes:
            for node_id_route, delivered_items_dict, _, _ in route:
                if node_id_route in delivered_quantities_at_nodes: # Ensure it's a customer node being tracked
                    for pt_id_item, qty_item in delivered_items_dict.items():
                        if pt_id_item in delivered_quantities_at_nodes[node_id_route]:
                            delivered_quantities_at_nodes[node_id_route][pt_id_item] += qty_item
                        else: # Delivered a product not demanded by this node
                            if debug_feasibility: print(f"DEBUG Feasibility Fail (Demand): Product {pt_id_item} delivered to {node_id_route} but not in its original demand profile.")
                            return False 
        
        for customer_node_obj in self.problem_instance.get_all_customer_nodes():
            node_id_check = customer_node_obj.id
            is_this_node_fully_serviced = True
            for pt_id_demand, demand_info_orig in customer_node_obj.demands.items():
                required_qty_orig = demand_info_orig["quantity"]
                delivered_qty_actual = delivered_quantities_at_nodes[node_id_check].get(pt_id_demand, 0)
                if delivered_qty_actual != required_qty_orig:
                    is_this_node_fully_serviced = False
                    if debug_feasibility: print(f"DEBUG Feasibility Fail (Demand): For {node_id_check}, product {pt_id_demand}. Need {required_qty_orig}, Got {delivered_qty_actual}.")
                    return False # Fails if any product demand for any customer is not met

            # Consistency check for self.unassigned_nodes
            if is_this_node_fully_serviced and node_id_check in self.unassigned_nodes:
                if debug_feasibility: print(f"DEBUG Feasibility Inconsistency: Node {node_id_check} is fully serviced but still in self.unassigned_nodes.")
                # This is an internal logic error, but not a feasibility failure of the route plan itself if demands are met.
                # For strictness, one might return False here, or just log it.
            elif not is_this_node_fully_serviced and node_id_check not in self.unassigned_nodes:
                if debug_feasibility: print(f"DEBUG Feasibility Inconsistency: Node {node_id_check} is NOT fully serviced but NOT in self.unassigned_nodes.")
                return False # This IS a feasibility failure for the solution state.


        # 2. Vehicle Capacities, Hard Time Windows, Warehouse Return Time
        for vehicle_idx, route_tuples in enumerate(self.routes):
            if not route_tuples:
                continue
            
            current_vehicle_load_volume = 0
            for _, delivered_items_stop, _, _ in route_tuples:
                for pt_id_item, qty_item in delivered_items_stop.items():
                    current_vehicle_load_volume += qty_item * self.problem_instance.product_order_types[pt_id_item].volume
            
            if current_vehicle_load_volume > self.problem_instance.vehicles[vehicle_idx].capacity_volume:
                if debug_feasibility: print(f"DEBUG Feasibility Fail (Capacity): Veh {vehicle_idx} capacity exceeded. Load {current_vehicle_load_volume}, Cap {self.problem_instance.vehicles[vehicle_idx].capacity_volume}")
                return False

            # Simulate route for timing
            current_node_id_sim = self.problem_instance.warehouse.id
            current_time_sim = self.problem_instance.warehouse.delivery_time_window[0]

            dist_to_first = self.problem_instance.get_distance(current_node_id_sim, route_tuples[0][0])
            current_time_sim += dist_to_first
            
            for i, (node_id_visited, _, _, _) in enumerate(route_tuples):
                arrival_time_at_node_sim = current_time_sim
                node_obj_visited = self.problem_instance.get_node(node_id_visited)
                
                effective_arrival_sim = arrival_time_at_node_sim
                if node_obj_visited.delivery_time_window and node_obj_visited.delivery_time_window[0] > arrival_time_at_node_sim:
                    effective_arrival_sim = node_obj_visited.delivery_time_window[0] # Wait if early

                if node_obj_visited.type == NODE_TYPE_STORE: # Hard TW
                    _, latest_time = node_obj_visited.delivery_time_window
                    if arrival_time_at_node_sim > latest_time:
                        if debug_feasibility: print(f"DEBUG Feasibility Fail (Hard TW): Store {node_id_visited}. Arrived {arrival_time_at_node_sim}, Latest {latest_time}")
                        return False
                
                departure_time_from_node_sim = effective_arrival_sim + node_obj_visited.service_time
                current_time_sim = departure_time_from_node_sim

                if i + 1 < len(route_tuples):
                    next_node_id_sim = route_tuples[i+1][0]
                    current_time_sim += self.problem_instance.get_distance(node_id_visited, next_node_id_sim)
                else: # Back to warehouse
                    current_time_sim += self.problem_instance.get_distance(node_id_visited, self.problem_instance.warehouse.id)

            # Check warehouse return time
            if current_time_sim > self.problem_instance.warehouse.delivery_time_window[1]:
                if debug_feasibility: print(f"DEBUG Feasibility Fail (Warehouse Return): Veh {vehicle_idx} returned {current_time_sim}, WH closes {self.problem_instance.warehouse.delivery_time_window[1]}")
                return False
        return True # All checks passed

    def __deepcopy__(self, memo):
        new_sol = Solution(self.problem_instance)
        new_sol.routes = copy.deepcopy(self.routes, memo)
        new_sol.cost = self.cost
        new_sol.unassigned_nodes = copy.deepcopy(self.unassigned_nodes, memo)
        return new_sol

    def __repr__(self):
        route_strs = []
        for i, route in enumerate(self.routes):
            if route: 
                stop_strs = []
                for r_node_id, r_items, _, _ in route:
                    item_detail = ", ".join([f"{pid}:{qty}" for pid, qty in r_items.items()])
                    stop_strs.append(f"{r_node_id}({item_detail})")
                route_strs.append(f"  Veh {i}: W0 -> " + " -> ".join(stop_strs) + " -> W0")
            else:
                route_strs.append(f"  Veh {i}: []")
        # Use a temporary call to is_feasible for the print string if needed, but avoid recursive prints
        is_f = self.is_feasible(debug_feasibility=False) # Call once for the repr
        return (f"Solution(Cost: {self.cost:.2f}, Feasible: {is_f})\n"
                f"Routes:\n" + "\n".join(route_strs) +
                f"\nUnassigned: {self.unassigned_nodes}")


class ALNS:
    def __init__(self, problem_instance, T_start=100, T_end=0.1, cooling_rate=0.999,
                 max_iter=1000, segment_len=50, reaction_factor=0.1,
                 score_new_global_best=16, score_better_than_current=8, score_accepted=4):
        self.problem_instance = problem_instance
        self.T_start_param = T_start 
        self.T_end = T_end
        self.cooling_rate = cooling_rate
        self.max_iter = max_iter
        self.segment_len = segment_len
        self.reaction_factor = reaction_factor

        self.scores = {
            "new_global_best": score_new_global_best,
            "better_than_current": score_better_than_current,
            "accepted_worse": score_accepted
        }
        
        self.destroy_operators = [self.random_removal] # Simplified to one for now
        self.repair_operators = [self.greedy_best_insertion_repair_revised] # Using the revised one

        self.destroy_weights = [1.0] * len(self.destroy_operators)
        self.destroy_scores = [0.0] * len(self.destroy_operators)
        self.destroy_counts = [0] * len(self.destroy_operators)

        self.repair_weights = [1.0] * len(self.repair_operators)
        self.repair_scores = [0.0] * len(self.repair_operators)
        self.repair_counts = [0] * len(self.repair_operators)

    def _select_operator(self, weights):
        # (Same as before)
        total_weight = sum(weights)
        if total_weight == 0:
            return random.randrange(len(weights))
        r = random.uniform(0, total_weight)
        upto = 0
        for i, w in enumerate(weights):
            if upto + w >= r:
                return i
            upto += w
        return len(weights) - 1


    def _update_weights(self):
        # (Same as before)
        for i in range(len(self.destroy_operators)):
            if self.destroy_counts[i] > 0:
                self.destroy_weights[i] = (1 - self.reaction_factor) * self.destroy_weights[i] + \
                                          self.reaction_factor * (self.destroy_scores[i] / self.destroy_counts[i])
            else: 
                self.destroy_weights[i] = max(0.1, (1 - self.reaction_factor) * self.destroy_weights[i])
            self.destroy_scores[i] = 0
            self.destroy_counts[i] = 0
        for i in range(len(self.repair_operators)):
            if self.repair_counts[i] > 0:
                self.repair_weights[i] = (1 - self.reaction_factor) * self.repair_weights[i] + \
                                         self.reaction_factor * (self.repair_scores[i] / self.repair_counts[i])
            else:
                self.repair_weights[i] = max(0.1, (1 - self.reaction_factor) * self.repair_weights[i])
            self.repair_scores[i] = 0
            self.repair_counts[i] = 0
            
    def initial_solution_greedy(self, sol_obj): # Pass the Solution object
        customer_nodes = self.problem_instance.get_all_customer_nodes()
        # Sort by latest delivery time (li), then by total demand volume (heuristic)
        sorted_nodes_to_assign = sorted(
            customer_nodes, 
            key=lambda n: (n.delivery_time_window[1] if n.delivery_time_window else float('inf'), 
                           -n.get_total_demand_volume()) # Process larger demands first for a given TW
        )

        sol_obj.unassigned_nodes = [node.id for node in customer_nodes] # Initialize/Reset

        print(f"DEBUG initial_solution_greedy: Starting. Nodes to assign: {[n.id for n in sorted_nodes_to_assign]}")

        for node_obj_to_assign in sorted_nodes_to_assign:
            node_id_current = node_obj_to_assign.id
            node_original_demands = node_obj_to_assign.demands 

            best_single_insertion_for_node = None 

            for v_idx in range(len(self.problem_instance.vehicles)):
                for p_idx in range(len(sol_obj.routes[v_idx]) + 1): # Try all positions
                    # Create a DEEP COPY of the CURRENT MAIN solution state to test this specific insertion
                    temp_sol_eval = copy.deepcopy(sol_obj) 
                    
                    required_volume_for_this_node_full_demand = node_obj_to_assign.get_total_demand_volume()
                    
                    current_vehicle_load_in_temp_sol = sum(
                        item_qty * self.problem_instance.product_order_types[item_id].volume
                        for _, delivered_items_dict, _, _ in temp_sol_eval.routes[v_idx] # Use route from temp_sol_eval
                        for item_id, item_qty in delivered_items_dict.items()
                    )
                    available_capacity_on_vehicle = self.problem_instance.vehicles[v_idx].capacity_volume - current_vehicle_load_in_temp_sol

                    if required_volume_for_this_node_full_demand <= available_capacity_on_vehicle:
                        items_to_deliver_this_visit = {
                            pt_id: d_info["quantity"] for pt_id, d_info in node_original_demands.items()
                        }
                        
                        new_stop_tuple = (node_id_current, items_to_deliver_this_visit, 0, 0) # Times are placeholders
                        temp_sol_eval.routes[v_idx].insert(p_idx, new_stop_tuple)

                        # **Crucial**: Update unassigned_nodes in temp_sol_eval correctly *before* calling is_feasible
                        if node_id_current in temp_sol_eval.unassigned_nodes:
                            temp_sol_eval.unassigned_nodes.remove(node_id_current) # Because we are attempting full delivery
                        
                        if temp_sol_eval.is_feasible(debug_feasibility=False): # Test on the modified temp_sol_eval
                            cost_of_this_temp_sol = temp_sol_eval.calculate_cost() # Cost of this complete temp solution
                            if best_single_insertion_for_node is None or cost_of_this_temp_sol < best_single_insertion_for_node[2]:
                                best_single_insertion_for_node = (v_idx, p_idx, cost_of_this_temp_sol)
                    # else: node's full demand doesn't fit volume-wise, skip for this simplified initial insertion.
            
            if best_single_insertion_for_node:
                v_idx_actual, p_idx_actual, _ = best_single_insertion_for_node
                
                items_for_actual_insertion = {
                    pt_id: d_info["quantity"] for pt_id, d_info in node_original_demands.items()
                }
                sol_obj.routes[v_idx_actual].insert(p_idx_actual, (node_id_current, items_for_actual_insertion, 0, 0))
                if node_id_current in sol_obj.unassigned_nodes: # Update main solution's unassigned list
                    sol_obj.unassigned_nodes.remove(node_id_current)
                print(f"DEBUG initial_solution_greedy: Inserted {node_id_current} (all demands) into Veh {v_idx_actual} Pos {p_idx_actual}.")
            else:
                print(f"DEBUG initial_solution_greedy: Could NOT find a feasible single insertion for all demands of {node_id_current}.")
                # Node remains in sol_obj.unassigned_nodes
        
        sol_obj.calculate_cost() # Calculate final cost of the constructed main solution
        print(f"Initial solution constructed. Cost: {sol_obj.cost:.2f}, Feasible: {sol_obj.is_feasible(debug_feasibility=False)}")
        return sol_obj # Return the modified Solution object

    def random_removal(self, current_sol, num_to_remove_factor=0.2):
        destroyed_sol = copy.deepcopy(current_sol)
        removed_nodes_info_list = [] 

        # Collect unique customer node IDs that are currently *in any route*
        # AND are *not* in the unassigned_nodes list (meaning they are considered serviced)
        candidate_nodes_for_removal = set()
        for route in destroyed_sol.routes:
            for stop_tuple in route:
                node_id_in_stop = stop_tuple[0]
                if node_id_in_stop not in destroyed_sol.unassigned_nodes: # Only consider removing serviced nodes
                     # Ensure it's a customer node
                    if self.problem_instance.get_node(node_id_in_stop).type != NODE_TYPE_WAREHOUSE:
                        candidate_nodes_for_removal.add(node_id_in_stop)
        
        if not candidate_nodes_for_removal:
            # print("DEBUG random_removal: No serviced customer nodes to remove.")
            return destroyed_sol, []

        num_nodes_to_actually_remove = min(len(candidate_nodes_for_removal), 
                                           max(1, int(len(candidate_nodes_for_removal) * num_to_remove_factor)))
        
        nodes_selected_for_full_removal = random.sample(list(candidate_nodes_for_removal), num_nodes_to_actually_remove)
        # print(f"DEBUG random_removal: Selected {nodes_selected_for_full_removal} for removal.")

        # This dictionary will store the *total* demands that were being serviced for the removed nodes
        total_demands_for_reinsertion = {node_id: {} for node_id in nodes_selected_for_full_removal}

        new_routes_after_removal = [[] for _ in destroyed_sol.routes]
        for v_idx, route_original in enumerate(destroyed_sol.routes):
            for stop_node_id, items_delivered_at_stop, arr_t, dep_t in route_original:
                if stop_node_id in nodes_selected_for_full_removal:
                    # Accumulate all items delivered to this node (across all its stops if it was split)
                    for pt_id, qty in items_delivered_at_stop.items():
                        current_sum = total_demands_for_reinsertion[stop_node_id].get(pt_id, 0)
                        total_demands_for_reinsertion[stop_node_id][pt_id] = current_sum + qty
                else: # This stop is not for a removed node, keep it
                    new_routes_after_removal[v_idx].append((stop_node_id, items_delivered_at_stop, arr_t, dep_t))
        
        destroyed_sol.routes = new_routes_after_removal

        for node_id_that_was_removed in nodes_selected_for_full_removal:
            # The demands to reinsert should be the node's *original total demands*,
            # not just what was on the routes (in case of partial service error).
            original_node_obj = self.problem_instance.get_node(node_id_that_was_removed)
            full_original_demands = {
                pt_id: d_info["quantity"] for pt_id, d_info in original_node_obj.demands.items()
            }
            removed_nodes_info_list.append({'id': node_id_that_was_removed, 'demands_removed': full_original_demands})
            
            if node_id_that_was_removed not in destroyed_sol.unassigned_nodes:
                destroyed_sol.unassigned_nodes.append(node_id_that_was_removed) # Mark as unassigned
        
        destroyed_sol.calculate_cost()
        # print(f"DEBUG random_removal: Destroyed sol cost {destroyed_sol.cost}, unassigned: {destroyed_sol.unassigned_nodes}")
        return destroyed_sol, removed_nodes_info_list

    def greedy_best_insertion_repair_revised(self, partial_sol_obj, removed_nodes_info_list):
        repaired_sol = copy.deepcopy(partial_sol_obj) # Start with the partially destroyed solution
        
        # Order of reinsertion can matter. Shuffle for some randomness.
        nodes_to_reinsert_ordered = [info['id'] for info in removed_nodes_info_list]
        random.shuffle(nodes_to_reinsert_ordered)

        # print(f"DEBUG repair: Starting. Nodes to reinsert: {nodes_to_reinsert_ordered}. Initial unassigned: {repaired_sol.unassigned_nodes}")

        for node_id_to_reinsert in nodes_to_reinsert_ordered:
            node_original_demands_map = self.problem_instance.get_node(node_id_to_reinsert).demands
            
            # Similar to initial solution, try to insert the node's full demand in one go
            best_single_insertion_for_repair = None

            for v_idx in range(len(self.problem_instance.vehicles)):
                for p_idx in range(len(repaired_sol.routes[v_idx]) + 1):
                    temp_sol_repair_eval = copy.deepcopy(repaired_sol)

                    required_volume_for_full_demand = self.problem_instance.get_node(node_id_to_reinsert).get_total_demand_volume()
                    
                    current_load_in_temp_repair_route = sum(
                        qty * self.problem_instance.product_order_types[pid].volume
                        for _, delivered, _, _ in temp_sol_repair_eval.routes[v_idx] for pid, qty in delivered.items()
                    )
                    available_cap_repair = self.problem_instance.vehicles[v_idx].capacity_volume - current_load_in_temp_repair_route

                    if required_volume_for_full_demand <= available_cap_repair:
                        items_for_this_repair_visit = {
                            pt_id: d_info["quantity"] for pt_id, d_info in node_original_demands_map.items()
                        }
                        
                        new_stop_repair = (node_id_to_reinsert, items_for_this_repair_visit, 0, 0)
                        temp_sol_repair_eval.routes[v_idx].insert(p_idx, new_stop_repair)

                        if node_id_to_reinsert in temp_sol_repair_eval.unassigned_nodes:
                            temp_sol_repair_eval.unassigned_nodes.remove(node_id_to_reinsert)

                        if temp_sol_repair_eval.is_feasible(debug_feasibility=False):
                            cost_temp_repair = temp_sol_repair_eval.calculate_cost()
                            if best_single_insertion_for_repair is None or cost_temp_repair < best_single_insertion_for_repair[2]:
                                best_single_insertion_for_repair = (v_idx, p_idx, cost_temp_repair)
            
            if best_single_insertion_for_repair:
                v_idx_act_rep, p_idx_act_rep, _ = best_single_insertion_for_repair
                items_for_actual_repair_insertion = {
                    pt_id: d_info["quantity"] for pt_id, d_info in node_original_demands_map.items()
                }
                repaired_sol.routes[v_idx_act_rep].insert(p_idx_act_rep, (node_id_to_reinsert, items_for_actual_repair_insertion, 0, 0))
                if node_id_to_reinsert in repaired_sol.unassigned_nodes:
                    repaired_sol.unassigned_nodes.remove(node_id_to_reinsert)
                # print(f"DEBUG repair: Re-inserted {node_id_to_reinsert} into Veh {v_idx_act_rep} Pos {p_idx_act_rep}.")
            # else:
                # print(f"DEBUG repair: Could NOT re-insert {node_id_to_reinsert}. It remains in unassigned: {repaired_sol.unassigned_nodes}")
        
        repaired_sol.calculate_cost()
        # print(f"DEBUG repair: Finished. Repaired sol cost {repaired_sol.cost}, unassigned: {repaired_sol.unassigned_nodes}")
        return repaired_sol


    def run(self):
        current_sol = Solution(self.problem_instance) # Create a blank solution
        print("Constructing initial solution...")
        current_sol = self.initial_solution_greedy(current_sol) # Populate it
        
        best_sol = copy.deepcopy(current_sol)
        
        h_temp_control = 0.05 
        T = h_temp_control * abs(current_sol.cost) if current_sol.cost != 0 else self.T_start_param # Use abs for safety
        T = max(T, self.T_start_param / 10, 1.0) # Ensure T is not excessively small, at least 1.0

        print(f"Starting ALNS. Initial solution cost: {current_sol.cost:.2f}, Feasible: {current_sol.is_feasible()}. Initial T: {T:.2f}")

        for i in range(self.max_iter):
            destroy_op_idx = self._select_operator(self.destroy_weights)
            repair_op_idx = self._select_operator(self.repair_weights)
            destroy_op = self.destroy_operators[destroy_op_idx]
            repair_op = self.repair_operators[repair_op_idx]

            # print(f"Iter {i+1}: Using Destroy: {destroy_op.__name__}, Repair: {repair_op.__name__}")

            temp_sol_destroyed, removed_nodes = destroy_op(current_sol)
            if not removed_nodes and current_sol.is_feasible(): # If nothing was removed from a feasible solution, skip repair
                new_sol = copy.deepcopy(current_sol)
            else:
                new_sol = repair_op(temp_sol_destroyed, removed_nodes)
            
            new_sol_is_feasible = new_sol.is_feasible() 
            current_sol_is_feasible = current_sol.is_feasible()

            score_for_op = 0
            accepted_this_iteration = False

            if new_sol_is_feasible:
                if current_sol_is_feasible:
                    if new_sol.cost < current_sol.cost: # Better feasible solution
                        current_sol = new_sol
                        score_for_op = self.scores["better_than_current"]
                        accepted_this_iteration = True
                        if new_sol.cost < best_sol.cost or not best_sol.is_feasible(): # New global best (either better cost or best was infeasible)
                            best_sol = new_sol
                            score_for_op = self.scores["new_global_best"]
                            # print(f"Iter {i+1}: New Feas Global Best! Cost {best_sol.cost:.2f}, T={T:.2f}")
                    elif T > 0 and random.random() < math.exp((current_sol.cost - new_sol.cost) / T) : # Accept worse feasible
                        current_sol = new_sol
                        score_for_op = self.scores["accepted_worse"]
                        accepted_this_iteration = True
                else: # New is feasible, current was not -> always accept new feasible
                    current_sol = new_sol
                    score_for_op = self.scores["new_global_best"] # Finding feasibility is a big win
                    accepted_this_iteration = True
                    # print(f"Iter {i+1}: Found Feasible Solution from Infeasible! Cost {current_sol.cost:.2f}")
                    best_sol = new_sol # This is the new best since previous was infeasible
            else: # new_sol is infeasible
                if not current_sol_is_feasible: # Both are infeasible
                    if new_sol.cost < current_sol.cost: # Accept if it reduces overall cost (incl. penalties)
                        current_sol = new_sol
                        # score_for_op = 1 # Small score for reducing infeasibility cost
                        accepted_this_iteration = True 
            
            if accepted_this_iteration:
                self.destroy_scores[destroy_op_idx] += score_for_op
                self.repair_scores[repair_op_idx] += score_for_op
            
            # Always count operator usage for weight updates
            self.destroy_counts[destroy_op_idx] += 1
            self.repair_counts[repair_op_idx] += 1


            if (i + 1) % self.segment_len == 0:
                self._update_weights()
            
            T = max(self.T_end, T * self.cooling_rate)
            
            if (i + 1) % 100 == 0: 
                 print(f"Iter {i+1}: Current Feas: {current_sol_is_feasible}, Cost {current_sol.cost:.2f} | "
                       f"Best Feas: {best_sol.is_feasible()}, Cost {best_sol.cost:.2f} | Temp {T:.2f}")

        print(f"\nALNS Finished. Best solution cost: {best_sol.cost:.2f}, Feasible: {best_sol.is_feasible(debug_feasibility=False)}")
        if not best_sol.is_feasible() and best_sol.cost < float('inf'): # Only print debug if it's not the huge initial penalty
            print("Final best solution is INFEASIBLE. Debugging why:")
            best_sol.is_feasible(debug_feasibility=True) 

        return best_sol

# --- Running the ALNS ---
print("\n--- Block 2: ALNS Framework Execution ---")
problem_inst = setup_simple_instance() # Total demand: S1(19) + S2(4) + M1(20) = 43. Each veh cap 50.

alns_solver = ALNS(problem_inst,
                   T_start=10000, # Higher initial T_start to allow more exploration if initial cost is high
                   T_end=0.1,
                   cooling_rate=0.995, # Standard cooling
                   max_iter=1000, # Increased iterations
                   segment_len=50,   # Standard segment length
                   reaction_factor=0.1,
                   score_new_global_best=16,
                   score_better_than_current=8,
                   score_accepted=4)

final_solution = alns_solver.run()
print("\nFinal Solution Details:")
print(final_solution)


--- Block 2: ALNS Framework Execution ---
Constructing initial solution...
DEBUG initial_solution_greedy: Starting. Nodes to assign: ['M1', 'S1', 'S2']
DEBUG initial_solution_greedy: Could NOT find a feasible single insertion for all demands of M1.
DEBUG initial_solution_greedy: Could NOT find a feasible single insertion for all demands of S1.
DEBUG initial_solution_greedy: Could NOT find a feasible single insertion for all demands of S2.
Initial solution constructed. Cost: 3000000.00, Feasible: False
Starting ALNS. Initial solution cost: 3000000.00, Feasible: False. Initial T: 150000.00
Iter 100: Current Feas: False, Cost 3000000.00 | Best Feas: False, Cost 3000000.00 | Temp 90865.57
Iter 200: Current Feas: False, Cost 3000000.00 | Best Feas: False, Cost 3000000.00 | Temp 55043.67
Iter 300: Current Feas: False, Cost 3000000.00 | Best Feas: False, Cost 3000000.00 | Temp 33343.83
Iter 400: Current Feas: False, Cost 3000000.00 | Best Feas: False, Cost 3000000.00 | Temp 20198.71
Iter 500